In [21]:
import pydantic
print(pydantic.VERSION)


2.5.3


Instance serialization to dict : 


In [1]:
from pydantic import BaseModel, ValidationError, model_validator
from datetime import date


class ScheduledCourse(BaseModel):
    dept : str
    course_name : str
    start_dt : date
    end_dt : date

    @model_validator(mode="before") # Using before the model is being instantiated
    @classmethod
    def validate_date(cls, data: dict):
        if data["start_dt"] > data["end_dt"]:
            raise ValueError("Start date must be before the End date")
        return data 

try:
    c1 = ScheduledCourse(
        dept="Alumina",
        course_name="Science",
        start_dt="2024-12-22",
        end_dt="2025-11-12"
    )
    print(c1)
except pydantic.ValidationError as e:
    print(e)



dept='Alumina' course_name='Science' start_dt=datetime.date(2024, 12, 22) end_dt=datetime.date(2025, 11, 12)


In [3]:
# Object instance to dictionary

c1.model_dump()

{'dept': 'Alumina',
 'course_name': 'Science',
 'start_dt': datetime.date(2024, 12, 22),
 'end_dt': datetime.date(2025, 11, 12)}

In [4]:
# If I want to exclude some items from serialization 

c1.model_dump(exclude={"dept", "course_name"})

{'start_dt': datetime.date(2024, 12, 22),
 'end_dt': datetime.date(2025, 11, 12)}

In [8]:
# If I want to include only specific  items for serialization 

c1.model_dump(include={"start_dt", "course_name"})

{'course_name': 'Science', 'start_dt': datetime.date(2024, 12, 22)}

=> Field Exclusions : 


exclude_unset : 

> parameter in Pydantic models allows you to exclude fields that have not been set (i.e., fields with their default value) when exporting a model to a dictionary. 

> This can be particularly useful when you want to generate partial updates or minimize the payload.

In [10]:

class User(BaseModel):
    id: int
    name: str
    email: str = None

user = User(id=1, name='Harry')

In [11]:
user.model_dump()

{'id': 1, 'name': 'Harry', 'email': None}

In [12]:
# Exporting the instance to a dictionary excluding unset fields
user.model_dump_json(exclude_unset=True)

'{"id":1,"name":"Harry"}'


exclude_defaults : 

> parameter in Pydantic models works similarly to exclude_unset, but it specifically excludes fields that have their default values when exporting a model to a dictionary. 

> This is useful when you want to avoid including fields that haven't been explicitly modified from their defaults.

In [24]:

class User(BaseModel):
    id: int
    name: str
    email: str = ""

user = User(id=1, name='Harry', email="")

In [25]:
user.model_dump()

{'id': 1, 'name': 'Harry', 'email': ''}

In [26]:
user.model_dump_json(exclude_defaults=True)

'{"id":1,"name":"Harry"}'

=> JSON Schema

In [1]:
from pydantic import BaseModel, ValidationError, model_validator
from datetime import date


class ScheduledCourse(BaseModel):
    dept : str = "?"
    course_name : str 
    start_dt : date = date.today()
    end_dt : date

    @model_validator(mode="after") 
    def validate_date(self):
        if self.start_dt > self.end_dt:
            raise ValueError("Start date must be before the End date")
        return self

# try:
#     c1 = ScheduledCourse(
#         dept="Alumina",
#         course_name="Science",
#         start_dt="2024-12-22",
#         end_dt="2025-11-12"
#     )
#     print(c1)
# except pydantic.ValidationError as e:
#     print(e)

In [2]:
ScheduledCourse.model_json_schema()

{'properties': {'dept': {'default': '?', 'title': 'Dept', 'type': 'string'},
  'course_name': {'title': 'Course Name', 'type': 'string'},
  'start_dt': {'default': '2024-08-04',
   'format': 'date',
   'title': 'Start Dt',
   'type': 'string'},
  'end_dt': {'format': 'date', 'title': 'End Dt', 'type': 'string'}},
 'required': ['course_name', 'end_dt'],
 'title': 'ScheduledCourse',
 'type': 'object'}

In [3]:
# EX.

from typing import List, Optional

class Item(BaseModel):
    name : str
    notes : Optional[str] = None

class Order(BaseModel):
    id : int
    items : List[Item]
    

In [13]:
import json
print(json.dumps(Order.model_json_schema(), indent=4))

{
    "$defs": {
        "Item": {
            "properties": {
                "name": {
                    "title": "Name",
                    "type": "string"
                },
                "notes": {
                    "anyOf": [
                        {
                            "type": "string"
                        },
                        {
                            "type": "null"
                        }
                    ],
                    "default": null,
                    "title": "Notes"
                }
            },
            "required": [
                "name"
            ],
            "title": "Item",
            "type": "object"
        }
    },
    "properties": {
        "id": {
            "title": "Id",
            "type": "integer"
        },
        "items": {
            "items": {
                "$ref": "#/$defs/Item"
            },
            "title": "Items",
            "type": "array"
        }
    },
    "required": [
     

=> Deserialization : 

> refers to the process of converting data formats like JSON into Python objects, 

In [25]:
from pydantic import BaseModel, EmailStr
from typing import Optional


class User(BaseModel):
    name : str
    age : int
    email : Optional[EmailStr] = None


In [26]:
json_user_data = '{"name": "King", "age":12, "email":"hello@gmail.com"}'


In [30]:
# converting JSON data into UserObj

u1 = User.model_validate_json(json_user_data)

In [31]:
u1.name

'King'

In [41]:
data_list = [
    '{"name": "Alice", "age": 30, "email": "alice@example.com"}',
    '{"name": "Bob", "email": "bob@example.com"}',
    '{"name": "Charlie", "age": 25}',
    '{"age": 22, "email": "david@example.com"}',
    '{"name": "Eve", "age": 28, "email": "eve@example.com"}'
]


for u in data_list:
    try:
        uobj = User.model_validate_json(u)
        print('VALID !! ', uobj.name)
    except ValidationError as e:
        print(e.errors()[0]['msg'])


VALID !!  Alice
Field required
VALID !!  Charlie
Field required
VALID !!  Eve
